In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sepidehmashayekhi","key":"dfd7b71aa4f75b4d225ad652dd14380b"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                              title                                               size  lastUpdated          downloadCount  
---------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
mkoklu42/acoustic-extinguisher-fire-dataset                      Acoustic Extinguisher Fire Dataset                 620KB  2022-02-09 17:59:52            163  
mkoklu42/durum-wheat-dataset                                     Durum Wheat Dataset                                983MB  2022-02-18 09:34:17             41  
mkoklu42/grapevine-leaves-image-dataset                          Grapevine Leaves Image Dataset                     109MB  2022-02-09 18:21:40            155  
mkoklu42/rice-image-dataset                                      Rice Image Dataset                                 222MB  2022-02-07 14:32:24            207  
mkoklu42/pumpkin-seeds-dataset          

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

 98% 1.00G/1.03G [00:07<00:00, 129MB/s]
100% 1.03G/1.03G [00:07<00:00, 148MB/s]


In [ ]:
! unzip asl-alphabet.zip

In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Input,Dropout,Conv2D,MaxPool2D
from keras.models import Model,Sequential
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
import os 

In [23]:
data_dir = "/content/asl_alphabet_train/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64, 64, 3)
n_classes = 29
val_frac = 0.1
batch_size = 64
data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_frac)
train_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size,
                                                     batch_size=batch_size,
                                                     shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size,
                                                   batch_size=batch_size, subset="validation")


Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [34]:
my_model = Sequential()
my_model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=target_dims))
my_model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
my_model.add(Flatten())
my_model.add(Dropout(0.5))
my_model.add(Dense(512, activation='relu'))
my_model.add(Dense(n_classes, activation='softmax'))

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [35]:
h=my_model.fit_generator(train_generator, epochs=5, validation_data=val_generator)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


1224/1224 [==============================] - 149s 121ms/step - loss: 1.2088 - accuracy: 0.6196 - val_loss: 0.6502 - val_accuracy: 0.7731
Epoch 2/5
1224/1224 [==============================] - 145s 118ms/step - loss: 0.2527 - accuracy: 0.9145 - val_loss: 0.5661 - val_accuracy: 0.8378
Epoch 3/5
1224/1224 [==============================] - 144s 117ms/step - loss: 0.1675 - accuracy: 0.9452 - val_loss: 0.6187 - val_accuracy: 0.8644
Epoch 4/5
1224/1224 [==============================] - 144s 118ms/step - loss: 0.1298 - accuracy: 0.9580 - val_loss: 0.5678 - val_accuracy: 0.8613
Epoch 5/5
1224/1224 [==============================] - 146s 119ms/step - loss: 0.1141 - accuracy: 0.9640 - val_loss: 0.3769 - val_accuracy: 0.8954


In [36]:
my_model.save('my_model.h5')